# Insert, Update and Delete data on S3 Datalake using Apache Hudi and Amazon EMR - Demo

## Table of Contents:

1. [Overview](#Overview)
2. [Copy On Write](#Copy-On-Write)<br>
    2.1 [Bulk Insert the Initial Dataset](#Bulk-Insert-the-Initial-Dataset)<br>
    2.2 [Batch Upsert some records](#Batch-Upsert-some-records)<br>
    2.3 [Deleting Records](#Deleting-Records)<br>
3. [Rollback](#Rollback)<br>
4. [Time travel with Hudi](#Time-travel-with-Hudi) 
5. [Merge on Read](#Merge-On-Read)<br>
    5.1 [Bulk Insert the Initial Dataset](#Bulk-Insert-the-Initial-Dataset)<br>
    5.2 [Batch Upsert some records](#Batch-Upsert-some-records)<br>
    5.3 [Compaction for MOR tables](#Compaction-for-MOR-tables) <br>
6. [Working with Partitioned Tables](#Working-with-Partitioned-Tables)

## Overview

This notebook demonstrates using PySpark on [Apache Hudi](https://aws.amazon.com/emr/features/hudi/) on Amazon EMR to insert/upsert/delete records to an S3 data lake.

Here are some good reference links to read later:

* [Apache Hudi concepts](https://hudi.apache.org/concepts.html)
* [How Hudi Works](https://docs.aws.amazon.com/emr/latest/ReleaseGuide/emr-hudi-how-it-works.html)

This notebook covers the following concepts when writing Copy-On-Write and Merge-On-Read tables to an S3 Datalake:

- Write Hudi Spark jobs in PySpark.
- Bulk Insert the Initial Dataset.
- Write a MultiKey Partitioned table as well as a Non-Partitioned table.
- Tune the Bulk Insert write performance as per expected number of target files.
- Sync the Hudi tables to the Hive/Glue Catalog.
- Upsert some records to a Hudi table.
- Delete from records from a Hudi table.
- Understand how Hudi Commit Retention policy works.
- Time travel with Hudi incremental tables using incremental and point-in-time queries
- Perform Insert/Upsert/Delete operations for Merge-on-Read table and understand the difference between MOR and COW tables

Let's start by initializing the Spark Session to connect this notebook to our Spark EMR cluster:

Note that the files hudi-spark-bundle.jar and spark-avro.jar are copied into HDFS.

hadoop fs -copyFromLocal /usr/lib/hudi/hudi-spark-bundle.jar hdfs:///user/hadoop/

hadoop fs -copyFromLocal /usr/lib/spark/external/lib/spark-avro.jar hdfs:///user/hadoop/

hadoop fs -copyFromLocal /usr/lib/spark/jars/httpclient-4.5.9.jar hdfs:///user/hadoop/

hadoop fs -copyFromLocal /usr/lib/spark/jars/httpcore-4.4.11.jar hdfs:///user/hadoop/

hadoop fs -copyFromLocal /usr/share/aws/aws-java-sdk/aws-java-sdk-bundle-1.12.31.jar hdfs:///user/hadoop/


In [ ]:
%%configure -f
{
    "conf":  { 
             "spark.jars":"hdfs:///user/hadoop/aws-java-sdk-bundle-1.12.31.jar,hdfs:///user/hadoop/httpcore-4.4.11.jar,hdfs:///user/hadoop/httpclient-4.5.9.jar,hdfs:////user/hadoop/hudi-spark-bundle.jar,hdfs:///user/hadoop/spark-avro.jar",
             "spark.sql.hive.convertMetastoreParquet":"false", 
             "spark.serializer":"org.apache.spark.serializer.KryoSerializer"
           } 
}

## Configuration

Following are the configuration items we will use in this demo

Make sure to point the target parameter to your S3 bucket. Replace youraccountID with your event engine AWS account.

In [ ]:
# REPLACE "youraccountID" with your event engine AWS account ID

config = {
    "table_name": "hudi_trips_table",
    "target": "s3://mrworkshop-youraccountID-dayone/hudi/hudi_trips_table",
    "primary_key": "trip_id",
    "sort_key": "tstamp",
    "commits_to_retain": "10"
}

The constants for Python to use:

In [ ]:
# General Constants
HUDI_FORMAT = "org.apache.hudi"
TABLE_NAME = "hoodie.table.name"
RECORDKEY_FIELD_OPT_KEY = "hoodie.datasource.write.recordkey.field"
PRECOMBINE_FIELD_OPT_KEY = "hoodie.datasource.write.precombine.field"
OPERATION_OPT_KEY = "hoodie.datasource.write.operation"
BULK_INSERT_OPERATION_OPT_VAL = "bulk_insert"
UPSERT_OPERATION_OPT_VAL = "upsert"
BULK_INSERT_PARALLELISM = "hoodie.bulkinsert.shuffle.parallelism"
UPSERT_PARALLELISM = "hoodie.upsert.shuffle.parallelism"
S3_CONSISTENCY_CHECK = "hoodie.consistency.check.enabled"
HUDI_CLEANER_POLICY = "hoodie.cleaner.policy"
KEEP_LATEST_COMMITS = "KEEP_LATEST_COMMITS"
HUDI_COMMITS_RETAINED = "hoodie.cleaner.commits.retained"
PAYLOAD_CLASS_OPT_KEY = "hoodie.datasource.write.payload.class"
EMPTY_PAYLOAD_CLASS_OPT_VAL = "org.apache.hudi.common.model.EmptyHoodieRecordPayload"

# Hive Constants
HIVE_SYNC_ENABLED_OPT_KEY="hoodie.datasource.hive_sync.enable"
HIVE_PARTITION_FIELDS_OPT_KEY="hoodie.datasource.hive_sync.partition_fields"
HIVE_ASSUME_DATE_PARTITION_OPT_KEY="hoodie.datasource.hive_sync.assume_date_partitioning"
HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY="hoodie.datasource.hive_sync.partition_extractor_class"
HIVE_TABLE_OPT_KEY="hoodie.datasource.hive_sync.table"

# Partition Constants
NONPARTITION_EXTRACTOR_CLASS_OPT_VAL="org.apache.hudi.hive.NonPartitionedExtractor"
MULTIPART_KEYS_EXTRACTOR_CLASS_OPT_VAL="org.apache.hudi.hive.MultiPartKeysValueExtractor"
KEYGENERATOR_CLASS_OPT_KEY="hoodie.datasource.write.keygenerator.class"
NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL="org.apache.hudi.keygen.NonpartitionedKeyGenerator"
COMPLEX_KEYGENERATOR_CLASS_OPT_VAL="org.apache.hudi.ComplexKeyGenerator"
PARTITIONPATH_FIELD_OPT_KEY="hoodie.datasource.write.partitionpath.field"

#Incremental Constants
VIEW_TYPE_OPT_KEY="hoodie.datasource.view.type"
BEGIN_INSTANTTIME_OPT_KEY="hoodie.datasource.read.begin.instanttime"
VIEW_TYPE_INCREMENTAL_OPT_VAL="incremental"
END_INSTANTTIME_OPT_KEY="hoodie.datasource.read.end.instanttime"

Functions to create JSON data and Spark dataframe from this data

In [ ]:
## Generates Data

from datetime import datetime

def get_json_data(start, count, dest):
    time_stamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    data = [{"trip_id": i, "tstamp": time_stamp, "route_id": chr(65 + (i % 10)), "destination": dest[i%10]} for i in range(start, start + count)]
    return data

# Creates the Dataframe
def create_json_df(spark, data):
    sc = spark.sparkContext
    return spark.read.json(sc.parallelize(data))


### Bulk Insert the Initial Dataset

Let's generate 2M records to load into our Data Lake:

In [ ]:
dest = ["Seattle", "New York", "New Jersey", "Los Angeles", "Las Vegas", "Tucson","Washington DC","Philadelphia","Miami","San Francisco"]
df1 = create_json_df(spark, get_json_data(0, 2000000, dest))
print(df1.count())
df1.show()

And write the data to S3:

In [ ]:
(df1.write.format(HUDI_FORMAT)
      .option(PRECOMBINE_FIELD_OPT_KEY, config["sort_key"])
      .option(RECORDKEY_FIELD_OPT_KEY, config["primary_key"])
      .option(TABLE_NAME, config['table_name'])
      .option(OPERATION_OPT_KEY, BULK_INSERT_OPERATION_OPT_VAL)
      .option(BULK_INSERT_PARALLELISM, 3)
      .option(S3_CONSISTENCY_CHECK, "true")
      .option(HIVE_TABLE_OPT_KEY,config['table_name'])
      .option(HIVE_SYNC_ENABLED_OPT_KEY,"true")
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,NONPARTITION_EXTRACTOR_CLASS_OPT_VAL)
      .option(KEYGENERATOR_CLASS_OPT_KEY,NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL)
      .mode("Overwrite")
      .save(config['target']))

Let's observe the number of files in S3. Expected number of files is 3 files as BULK_INSERT_PARALLELISM is set to 3. 

REPLACE youraccountID with event engine AWS account ID 

```

$ aws s3 ls s3://mrworkshop-youraccountID-dayone/hudi/hudi_trips_table/ --summarize --human-readable
                           PRE .hoodie/
2022-03-01 05:28:13    0 Bytes .hoodie_$folder$
2022-03-01 05:28:25   93 Bytes .hoodie_partition_metadata
2022-03-01 05:28:33    4.6 MiB 387ba1b1-854d-4add-b294-ccbdeb1d61d2-0_1-8-132_20220301052810.parquet
2022-03-01 05:28:34    4.5 MiB a4e39bef-3ff1-4c0e-a4d2-853575468021-0_2-8-133_20220301052810.parquet
2022-03-01 05:28:34    4.8 MiB b51530d5-d7a9-41c7-9e3e-63a7a6ec8098-0_0-8-131_20220301052810.parquet

Total Objects: 5
   Total Size: 13.9 MiB

```

```

Let's inspect the table created and query the data.

In [ ]:
spark.sql("show tables").show(20,False)

Check the Glue data catalog for a new Hudi table called hudi_trips_table. Check the schema and notice the new columns added by Hudi to keep track of commits and filenames

In [ ]:
df2=spark.read.format(HUDI_FORMAT).load(config["target"]+"/*")
df2.count()
df2.show(20,False)

We can query the Hive table as well:

In [ ]:
spark.sql("select count(*) from "+config['table_name']).show(20,False)

### Batch Upsert some records

Let's modify a few records:

In [ ]:
spark.sql("select trip_id, route_id, destination, tstamp from "+config['table_name'] +" where trip_id between 1000000 and 1000009").show(20,False)

In [ ]:
upsert_dest = ["Boston", "Boston", "Boston", "Boston", "Boston","Boston","Boston","Boston","Boston","Boston"]
df3 = create_json_df(spark, get_json_data(1000000, 10, upsert_dest))
df3.count()

In [ ]:
df3.select("trip_id","route_id","tstamp","destination").show()

We have changed the destination and timestamp for trip IDs 1000000 to 1000010. Now, let's upsert the changes to S3. Note that the operation now is "Upsert" as opposed to BulkInsert for the initial load:

```
      .option(OPERATION_OPT_KEY, UPSERT_OPERATION_OPT_VAL)

```

In [ ]:
(df3.write.format(HUDI_FORMAT)
      .option(PRECOMBINE_FIELD_OPT_KEY, config["sort_key"])
      .option(RECORDKEY_FIELD_OPT_KEY, config["primary_key"])
      .option(TABLE_NAME, config['table_name'])
      .option(OPERATION_OPT_KEY, UPSERT_OPERATION_OPT_VAL)
      .option(UPSERT_PARALLELISM, 20)
      .option(S3_CONSISTENCY_CHECK, "true")
      .option(HUDI_CLEANER_POLICY, KEEP_LATEST_COMMITS)
      .option(HUDI_COMMITS_RETAINED,config["commits_to_retain"])
      .option(HIVE_TABLE_OPT_KEY,config['table_name'])
      .option(HIVE_SYNC_ENABLED_OPT_KEY,"true")
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,NONPARTITION_EXTRACTOR_CLASS_OPT_VAL)
      .option(KEYGENERATOR_CLASS_OPT_KEY,NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL)  
      .mode("Append")
      .save(config['target']))

In [ ]:
spark.sql("select trip_id, route_id, tstamp, destination from "+config['table_name'] +" where trip_id between 999996 and 1000013").show(50,False)

Now lets check the commit. 

REPLACE youraccountID with event engine AWS account ID.

```
$ aws s3 ls s3://mrworkshop-youraccountID-dayone/hudi/hudi_trips_table/ --summarize --human-readable
                           PRE .hoodie/
2022-03-01 05:28:13    0 Bytes .hoodie_$folder$
2022-03-01 05:28:25   93 Bytes .hoodie_partition_metadata
2022-03-01 05:28:33    4.6 MiB 387ba1b1-854d-4add-b294-ccbdeb1d61d2-0_1-8-132_20220301052810.parquet
2022-03-01 05:28:34    4.5 MiB a4e39bef-3ff1-4c0e-a4d2-853575468021-0_2-8-133_20220301052810.parquet
2022-03-01 05:31:37    4.8 MiB b51530d5-d7a9-41c7-9e3e-63a7a6ec8098-0_0-55-728_20220301053126.parquet
2022-03-01 05:28:34    4.8 MiB b51530d5-d7a9-41c7-9e3e-63a7a6ec8098-0_0-8-131_20220301052810.parquet

Total Objects: 6
   Total Size: 18.7 MiB

```

Now that we upserted some records, let us try to insert 10 new records into the table. We will use same upsert option. As primary keys 2000000 to 2000009 do not exist in the table, the records will be inserted. 

In [ ]:
insert_dest = ["Syracuse", "Syracuse", "Syracuse", "Syracuse", "Syracuse", "Syracuse", "Syracuse", "Syracuse", "Syracuse", "Syracuse"]
df5 = create_json_df(spark, get_json_data(2000000, 10, insert_dest))
df5.count()
df5.show()

In [ ]:
(df5.write.format(HUDI_FORMAT)
      .option(PRECOMBINE_FIELD_OPT_KEY, config["sort_key"])
      .option(RECORDKEY_FIELD_OPT_KEY, config["primary_key"])
      .option(TABLE_NAME, config['table_name'])
      .option(OPERATION_OPT_KEY, UPSERT_OPERATION_OPT_VAL)
      .option(UPSERT_PARALLELISM, 20)
      .option(S3_CONSISTENCY_CHECK, "true")
      .option(HUDI_CLEANER_POLICY, KEEP_LATEST_COMMITS)
      .option(HUDI_COMMITS_RETAINED,config["commits_to_retain"])
      .option(HIVE_TABLE_OPT_KEY,config['table_name'])
      .option(HIVE_SYNC_ENABLED_OPT_KEY,"true")
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,NONPARTITION_EXTRACTOR_CLASS_OPT_VAL)
      .option(KEYGENERATOR_CLASS_OPT_KEY,NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL)  
      .mode("Append")
      .save(config['target']))

In [ ]:
df6=spark.read.format(HUDI_FORMAT).load(config["target"]+"/*")
df6.count()

In [ ]:
spark.sql("select trip_id, route_id, tstamp, destination from "+config['table_name'] +" where trip_id > 1999996").show(50,False)

We can observe that the records are updated.

## Deleting Records.

Apache Hudi supports implementing two types of deletes on data stored in Hudi datasets, by enabling the user to specify a different record payload implementation.

* **Soft Deletes** : With soft deletes, user wants to retain the key but just null out the values for all other fields. This can be simply achieved by ensuring the appropriate fields are nullable in the dataset schema and simply upserting the dataset after setting these fields to null.
    
* **Hard Deletes** : A stronger form of delete is to physically remove any trace of the record from the dataset. 

Let's now execute some hard delete operations on our dataset which will remove the records from our dataset.

Let's delete the 10 records with the "Syracuse" destination we added to the table. Note that the only change is the single line that set the hoodie.datasource.write.payload.class to org.apache.hudi.EmptyHoodieRecordPayload to delete the records.

```
.option(PAYLOAD_CLASS_OPT_KEY, EMPTY_PAYLOAD_CLASS_OPT_VAL)
```

In [ ]:
df5.show()

In [ ]:
(df5.write.format(HUDI_FORMAT)
      .option(PRECOMBINE_FIELD_OPT_KEY, config["sort_key"])
      .option(RECORDKEY_FIELD_OPT_KEY, config["primary_key"])
      .option(TABLE_NAME, config['table_name'])
      .option(OPERATION_OPT_KEY, UPSERT_OPERATION_OPT_VAL)
      .option(UPSERT_PARALLELISM, 20)
      .option(S3_CONSISTENCY_CHECK, "true")
      .option(HUDI_CLEANER_POLICY, KEEP_LATEST_COMMITS)
      .option(HUDI_COMMITS_RETAINED,config["commits_to_retain"])
      .option(HIVE_TABLE_OPT_KEY,config['table_name'])
      .option(HIVE_SYNC_ENABLED_OPT_KEY,"true")
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,NONPARTITION_EXTRACTOR_CLASS_OPT_VAL)
      .option(KEYGENERATOR_CLASS_OPT_KEY,NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL)
      .option(PAYLOAD_CLASS_OPT_KEY, EMPTY_PAYLOAD_CLASS_OPT_VAL)
      .mode("Append")
      .save(config['target']))

In [ ]:
spark.sql("select trip_id, route_id, tstamp, destination from "+config['table_name'] +" where trip_id > 1999996").show(20,False)

We can observe that the records > 2000000 no longer exist in our table.

Let's observe the number of files in S3. Expected : 6 files (initial files (3) + one upsert + one insert + one delete = 6)

REPLACE youraccountID with event engine AWS account ID 

```

$ aws s3 ls s3://mrworkshop-youraccountID-dayone/hudi/hudi_trips_table/ --summarize --human-readable
                           PRE .hoodie/
2022-03-01 05:28:13    0 Bytes .hoodie_$folder$
2022-03-01 05:28:25   93 Bytes .hoodie_partition_metadata
2022-03-01 05:28:33    4.6 MiB 387ba1b1-854d-4add-b294-ccbdeb1d61d2-0_1-8-132_20220301052810.parquet
2022-03-01 05:33:09    4.5 MiB a4e39bef-3ff1-4c0e-a4d2-853575468021-0_0-141-1576_20220301053259.parquet
2022-03-01 05:32:44    4.5 MiB a4e39bef-3ff1-4c0e-a4d2-853575468021-0_0-98-1180_20220301053232.parquet
2022-03-01 05:28:34    4.5 MiB a4e39bef-3ff1-4c0e-a4d2-853575468021-0_2-8-133_20220301052810.parquet
2022-03-01 05:31:37    4.8 MiB b51530d5-d7a9-41c7-9e3e-63a7a6ec8098-0_0-55-728_20220301053126.parquet
2022-03-01 05:28:34    4.8 MiB b51530d5-d7a9-41c7-9e3e-63a7a6ec8098-0_0-8-131_20220301052810.parquet

Total Objects: 8
   Total Size: 27.8 MiB

```

In our example, we set number of commits to retain as 10. So, maximum only 10 new files can be created on top of our bulk insert files. i.e., 13 files in total. If we had set the commits_to_retain as 2, the number of files created will not increase beyond initial files(3) + commits_to_retain(2) = 5 files. This is because Hudi Cleaning Policy will delete older files when writing based on the commit retain policy.

## Rollback

Let's say we want to roll back the last delete we made. Run the below command on EMR leader node session (using Session Manager or SSH).

/usr/lib/hudi/cli/bin/hudi-cli.sh

REPLACE youraccountID with event engine AWS account ID.

connect --path s3://mrworkshop-youraccountID-dayone/hudi/hudi_trips_table/

```
hudi:hudi_trips_table->commits show
2022-03-01 05:34:49,072 INFO timeline.HoodieActiveTimeline: Loaded instants upto : Option{val=[20220301053259__commit__COMPLETED]}
2022-03-01 05:34:49,096 INFO s3n.S3NativeFileSystem: Opening 's3://mrworkshop-youraccountID-dayone/hudi/hudi_trips_table/.hoodie/20220301053259.commit' for reading
2022-03-01 05:34:49,456 INFO s3n.S3NativeFileSystem: Opening 's3://mrworkshop-youraccountID-dayone/hudi/hudi_trips_table/.hoodie/20220301053232.commit' for reading
2022-03-01 05:34:49,563 INFO s3n.S3NativeFileSystem: Opening 's3://mrworkshop-youraccountID-dayone/hudi/hudi_trips_table/.hoodie/20220301053126.commit' for reading
2022-03-01 05:34:49,632 INFO s3n.S3NativeFileSystem: Opening 's3://mrworkshop-youraccountID-dayone/hudi/hudi_trips_table/.hoodie/20220301052810.commit' for reading
╔════════════════╤═════════════════════╤═══════════════════╤═════════════════════╤══════════════════════════╤═══════════════════════╤══════════════════════════════╤══════════════╗
║ CommitTime     │ Total Bytes Written │ Total Files Added │ Total Files Updated │ Total Partitions Written │ Total Records Written │ Total Update Records Written │ Total Errors ║
╠════════════════╪═════════════════════╪═══════════════════╪═════════════════════╪══════════════════════════╪═══════════════════════╪══════════════════════════════╪══════════════╣
║ 20220301053259 │ 4.5 MB              │ 0                 │ 1                   │ 1                        │ 650365                │ 0                            │ 0            ║
╟────────────────┼─────────────────────┼───────────────────┼─────────────────────┼──────────────────────────┼───────────────────────┼──────────────────────────────┼──────────────╢
║ 20220301053232 │ 4.5 MB              │ 0                 │ 1                   │ 1                        │ 650375                │ 0                            │ 0            ║
╟────────────────┼─────────────────────┼───────────────────┼─────────────────────┼──────────────────────────┼───────────────────────┼──────────────────────────────┼──────────────╢
║ 20220301053126 │ 4.8 MB              │ 0                 │ 1                   │ 1                        │ 682498                │ 10                           │ 0            ║
╟────────────────┼─────────────────────┼───────────────────┼─────────────────────┼──────────────────────────┼───────────────────────┼──────────────────────────────┼──────────────╢
║ 20220301052810 │ 13.9 MB             │ 3                 │ 0                   │ 1                        │ 2000000               │ 0                            │ 0            ║
╚════════════════╧═════════════════════╧═══════════════════╧═════════════════════╧══════════════════════════╧═══════════════════════╧══════════════════════════════╧══════════════╝
```

REPLACE the --commit value with the first CommitTime returned by the above command. 

hudi:hudi_trips_table->commit rollback --commit 20220301053259
    
Now let us check what happened to the records we deleted earlier.

In [ ]:
spark.sql("select trip_id, route_id, tstamp, destination from "+config['table_name'] +" where trip_id > 1999996").show(20,False)

### Time Travel with Hudi 

Now, let us time travel with Hudi (query previous commits) with incremental and point-in-time queries

First, lets check out incremental queries

In EMR cluster, let us check Hudi CLI and check how commits look right now.

/usr/lib/hudi/cli/bin/hudi-cli.sh

REPLACE youraccountID with event engine AWS account ID 

connect --path s3://mrworkshop-youraccountID-dayone/hudi/hudi_trips_table/

commits show

```
╔════════════════╤═════════════════════╤═══════════════════╤═════════════════════╤══════════════════════════╤═══════════════════════╤══════════════════════════════╤══════════════╗
║ CommitTime     │ Total Bytes Written │ Total Files Added │ Total Files Updated │ Total Partitions Written │ Total Records Written │ Total Update Records Written │ Total Errors ║
╠════════════════╪═════════════════════╪═══════════════════╪═════════════════════╪══════════════════════════╪═══════════════════════╪══════════════════════════════╪══════════════╣
║ 20220301053232 │ 4.5 MB              │ 0                 │ 1                   │ 1                        │ 650375                │ 0                            │ 0            ║
╟────────────────┼─────────────────────┼───────────────────┼─────────────────────┼──────────────────────────┼───────────────────────┼──────────────────────────────┼──────────────╢
║ 20220301053126 │ 4.8 MB              │ 0                 │ 1                   │ 1                        │ 682498                │ 10                           │ 0            ║
╟────────────────┼─────────────────────┼───────────────────┼─────────────────────┼──────────────────────────┼───────────────────────┼──────────────────────────────┼──────────────╢
║ 20220301052810 │ 13.9 MB             │ 3                 │ 0                   │ 1                        │ 2000000               │ 0                            │ 0            ║
╚════════════════╧═════════════════════╧═══════════════════╧═════════════════════╧══════════════════════════╧═══════════════════════╧══════════════════════════════╧══════════════╝
```

Now, lets check out point-in-time queries. i.e., query from a specific commit 

In [ ]:
commits = spark.sql("select distinct(_hoodie_commit_time) as commitTime from  hudi_trips_table order by commitTime").collect()
print("commits: ")
for elem in commits: print (elem) 

In [ ]:
#Lets check out the first state. i.e., before we updated the trip to Boston

beginTime = "000" #Represents all commits > this time.
endTime = commits[-3][0] # first commit
#print("end time: "+endTime)

# incrementally query data
incViewDF = spark.read.format("org.apache.hudi") \
                   .option(VIEW_TYPE_OPT_KEY, VIEW_TYPE_INCREMENTAL_OPT_VAL) \
                   .option(BEGIN_INSTANTTIME_OPT_KEY, beginTime) \
                   .option(END_INSTANTTIME_OPT_KEY, endTime) \
                   .load(config["target"]) 

#incViewDF.show(5)
incViewDF.registerTempTable("hudi_incr_table")

In [ ]:
spark.sql("select `_hoodie_commit_time`, trip_id, route_id, destination, tstamp from  hudi_incr_table where trip_id between 999996 and 1000013").show()

Now we can see that the records from 1000000 to 1000009 display first state of our table before we upserted the trip destination to Boston

### Merge On Read 

The default table type is Copy-On-Write which is best suited for read-heavy workloads with modest writes. Copy-On-Write creates commit files with original data + the new changes during writing itself. While this increases latency on writes, this set up makes it more manageable for faster read.

For near real-time applications that mandate quick upserts, MERGE_ON_READ table type would be better suited. MOR table stores incoming upserts for each file group, onto a row based delta log (In Avro file format). This log is then merged with the existing Parquet file using a a compactor during reads.

In [ ]:
# REPLACE youraccountID with event engine AWS account ID 

config = {
    "table_name": "hudi_mor_trips_table",
    "table_name_rt": "hudi_mor_trips_table_rt",
    "target": "s3://mrworkshop-youraccountID-dayone/hudi/hudi_mor_trips_table",
    "primary_key": "trip_id",
    "sort_key": "tstamp",
    "commits_to_retain": "2",
}

In [ ]:
STORAGE_TYPE_OPT_KEY="hoodie.datasource.write.storage.type"
COMPACTION_INLINE_OPT_KEY="hoodie.compact.inline"
COMPACTION_MAX_DELTA_COMMITS_OPT_KEY="hoodie.compact.inline.max.delta.commits"

In [ ]:
mor_dest = ["Seattle", "New York", "New Jersey", "Los Angeles", "Las Vegas", "Tucson","Washington DC","Philadelphia","Miami","San Francisco"]
df2 = create_json_df(spark, get_json_data(0, 2000000, mor_dest))
df2.show()

Bulk insert will take the same time as COW as this is the first write 

In [ ]:
(df2.write.format(HUDI_FORMAT)
      .option(PRECOMBINE_FIELD_OPT_KEY, config["sort_key"])
      .option(RECORDKEY_FIELD_OPT_KEY, config["primary_key"])
      .option(TABLE_NAME, config['table_name'])
      .option(OPERATION_OPT_KEY, BULK_INSERT_OPERATION_OPT_VAL)
      .option(BULK_INSERT_PARALLELISM, 3)
      .option(S3_CONSISTENCY_CHECK, "true")
      .option(HIVE_TABLE_OPT_KEY,config['table_name'])
      .option(HIVE_SYNC_ENABLED_OPT_KEY,"true")
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,NONPARTITION_EXTRACTOR_CLASS_OPT_VAL)
      .option(KEYGENERATOR_CLASS_OPT_KEY,NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL)
      .option(STORAGE_TYPE_OPT_KEY, "MERGE_ON_READ")
      .option(COMPACTION_INLINE_OPT_KEY, "false")
      .option(COMPACTION_MAX_DELTA_COMMITS_OPT_KEY, "1")
      .mode("Overwrite")
      .save(config['target']))

### Verify the number of files 

Let us check the contents of S3 path. Bulk insert operation on Copy-On-Write and Merge-On-Read tables is identical in terms of performance. 

REPLACE youraccountID with event engine AWS account ID.

```
$ aws s3 ls s3://mrworkshop-youraccountID-dayone/hudi/hudi_mor_trips_table/
                           PRE .hoodie/
2022-03-01 05:40:47          0 .hoodie_$folder$
2022-03-01 05:40:53         93 .hoodie_partition_metadata
2022-03-01 05:41:00    4668560 426b8846-8e9f-470b-9b46-859d6c32877b-0_2-184-2214_20220301054046.parquet
2022-03-01 05:41:01    4841603 4eeaa768-d945-42a9-8f5d-aab7abacb9a2-0_1-184-2213_20220301054046.parquet
2022-03-01 05:41:01    4974502 4fd9dbe9-2575-4b61-9808-787b6cf8880f-0_0-184-2212_20220301054046.parquet

```

Notice the delta commits 

REPLACE youraccountID with event engine AWS account ID.

```
$ aws s3 ls s3://mrworkshop-youraccountID-dayone/hudi/hudi_mor_trips_table/.hoodie/
                           PRE .aux/
2022-03-01 05:40:48          0 .aux_$folder$
2022-03-01 05:40:47          0 .temp_$folder$
2022-03-01 05:41:01       3508 20220301054046.deltacommit
2022-03-01 05:40:49          0 20220301054046.deltacommit.inflight
2022-03-01 05:40:49          0 20220301054046.deltacommit.requested
2022-03-01 05:40:47          0 archived_$folder$
2022-03-01 05:40:48        370 hoodie.properties

```

This is the first commit 

Now let us try to upsert some records into MOR table

In [ ]:
upsert_dest = ["San Diego", "San Diego", "San Diego", "San Diego", "San Diego","San Diego","San Diego","San Diego","San Diego","San Diego"]
df3 = create_json_df(spark, get_json_data(1000000, 10, upsert_dest))
df3.count()
df3.show()

In [ ]:
(df3.write.format(HUDI_FORMAT)
      .option(PRECOMBINE_FIELD_OPT_KEY, config["sort_key"])
      .option(RECORDKEY_FIELD_OPT_KEY, config["primary_key"])
      .option(TABLE_NAME, config['table_name'])
      .option(OPERATION_OPT_KEY, UPSERT_OPERATION_OPT_VAL)
      .option(UPSERT_PARALLELISM, 20)
      .option(S3_CONSISTENCY_CHECK, "true")
      .option(HUDI_CLEANER_POLICY, KEEP_LATEST_COMMITS)
      .option(HUDI_COMMITS_RETAINED,config["commits_to_retain"])
      .option(HIVE_TABLE_OPT_KEY,config['table_name'])
      .option(HIVE_SYNC_ENABLED_OPT_KEY,"true")
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,NONPARTITION_EXTRACTOR_CLASS_OPT_VAL)
      .option(KEYGENERATOR_CLASS_OPT_KEY,NONPARTITIONED_KEYGENERATOR_CLASS_OPT_VAL)  
      .option(STORAGE_TYPE_OPT_KEY, "MERGE_ON_READ")
      .option(COMPACTION_INLINE_OPT_KEY, "false")
      .option(COMPACTION_MAX_DELTA_COMMITS_OPT_KEY, "1")
      .mode("Append")
      .save(config['target']))

In [ ]:
spark.sql("select trip_id, route_id, destination, tstamp from "+config['table_name']+ "_ro where trip_id between 999996 and 1000010").show(20,False)

Lets query the real-time table

In [ ]:
spark.sql("select trip_id, route_id, destination, tstamp from "+ config['table_name_rt'] + " where trip_id between 999996 and 1000010").show(20,False)

Check the S3 path again. There is no change in number of Parquet files after upsert operation unlike Copy-On-Write tables. 

REPLACE youraccountID with event engine AWS account ID.

```
$ aws s3 ls s3://mrworkshop-youraccountID-dayone/hudi/hudi_mor_trips_table/
                           PRE .hoodie/
2022-03-01 05:42:04       2071 .4fd9dbe9-2575-4b61-9808-787b6cf8880f-0_20220301054046.log.1_0-218-2541
2022-03-01 05:40:47          0 .hoodie_$folder$
2022-03-01 05:40:53         93 .hoodie_partition_metadata
2022-03-01 05:41:00    4668560 426b8846-8e9f-470b-9b46-859d6c32877b-0_2-184-2214_20220301054046.parquet
2022-03-01 05:41:01    4841603 4eeaa768-d945-42a9-8f5d-aab7abacb9a2-0_1-184-2213_20220301054046.parquet
2022-03-01 05:41:01    4974502 4fd9dbe9-2575-4b61-9808-787b6cf8880f-0_0-184-2212_20220301054046.parquet

```

Now, let us compact using Hudi CLI 

On EMR:

```
/usr/lib/hudi/cli/bin/hudi-cli.sh

REPLACE youraccountID with event engine AWS account ID.

hudi->connect --path s3://mrworkshop-youraccountID-dayone/hudi/hudi_mor_trips_table/

hudi:hudi_mor_trips_table->compactions show all
╔═════════════════════════╤═══════╤═══════════════════════════════╗
║ Compaction Instant Time │ State │ Total FileIds to be Compacted ║
╠═════════════════════════╧═══════╧═══════════════════════════════╣
║ (empty)                                                         ║
╚═════════════════════════════════════════════════════════════════╝

```

Notice there are no compactions for our mor table. Let us manually schedule and run a compaction. 

```

hudi:hudi_mor_trips_table->compaction schedule --hoodieConfigs hoodie.datasource.hive_sync.partition_extractor_class=com.uber.hoodie.NonpartitionedKeyGenerator,hoodie.compact.inline.max.delta.commits=1

Attempted to schedule compaction for 20220301054343

Exit the Hudi CLI (Ctrl+C) and re-enter 

/usr/lib/hudi/cli/bin/hudi-cli.sh

REPLACE youraccountID with event engine AWS account ID.

hudi:hudi_mor_trips_table->connect --path s3://mrworkshop-youraccountID-dayone/hudi/hudi_mor_trips_table/

hudi:hudi_mor_trips_table->compactions show all
2022-03-01 05:44:42,756 INFO timeline.HoodieActiveTimeline: Loaded instants upto : Option{val=[==>20220301054343__compaction__REQUESTED]}
2022-03-01 05:44:42,779 INFO s3n.S3NativeFileSystem: Opening 's3://mrworkshop-youraccountID-dayone/hudi/hudi_mor_trips_table/.hoodie/.aux/20220301054343.compaction.requested' for reading
╔═════════════════════════╤═══════════╤═══════════════════════════════╗
║ Compaction Instant Time │ State     │ Total FileIds to be Compacted ║
╠═════════════════════════╪═══════════╪═══════════════════════════════╣
║ 20220301054343          │ REQUESTED │ 1                             ║
╚═════════════════════════╧═══════════╧═══════════════════════════════╝

REPLACE youraccountID with your event engine AWS account ID.
REPLACE --compactionInstant value with what you received when you ran "compactions show all"

hudi:hudi_mor_trips_table->compaction run --parallelism 10 --sparkMemory 4G --retry 2 --schemaFilePath s3://mrworkshop-youraccountID-dayone/schema/schema.avsc --compactionInstant 20220301054343

Compaction successfully completed for 20220301054343

```

You can also schedule a compaction using "compaction schedule" option. 

Now, if we check the S3 path, we will see the delta commit has been merged into a new file 

REPLACE youraccountID with event engine AWS account ID.

```
$ aws s3 ls s3://mrworkshop-youraccountID-dayone/hudi/hudi_mor_trips_table/
                           PRE .hoodie/
2022-03-01 05:42:04       2071 .4fd9dbe9-2575-4b61-9808-787b6cf8880f-0_20220301054046.log.1_0-218-2541
2022-03-01 05:40:47          0 .hoodie_$folder$
2022-03-01 05:40:53         93 .hoodie_partition_metadata
2022-03-01 05:41:00    4668560 426b8846-8e9f-470b-9b46-859d6c32877b-0_2-184-2214_20220301054046.parquet
2022-03-01 05:41:01    4841603 4eeaa768-d945-42a9-8f5d-aab7abacb9a2-0_1-184-2213_20220301054046.parquet
2022-03-01 05:48:02    5405713 4fd9dbe9-2575-4b61-9808-787b6cf8880f-0_0-0-0_20220301054343.parquet
2022-03-01 05:41:01    4974502 4fd9dbe9-2575-4b61-9808-787b6cf8880f-0_0-184-2212_20220301054046.parquet

```


## Working with Partitioned Tables

Let's do the same thing with Partitioned Tables. For the sake of this demo, we will be making route_id as partition field. You can also have a nested partition structure like yyyy/mm/dd which is more common

In [ ]:
# REPLACE youraccountID with event engine AWS account ID 

config = {
    "table_name": "hudi_partitioned_trips_table",
    "target": "s3://mrworkshop-youraccountID-dayone/hudi/hudi_partitioned_trips_table",
    "primary_key": "trip_id",
    "sort_key": "tstamp",
    "commits_to_retain": "2",
    "partition_keys" : "route_id"
}

Let's generate the data:

In [ ]:
part_dest = ["Seattle", "New York", "New Jersey", "Los Angeles", "Las Vegas", "Tucson","Washington DC","Philadelphia","Miami","San Francisco"]
df1 = create_json_df(spark, get_json_data(0, 2000000, part_dest))

We add the partitionKey column to the dataframe.

In [ ]:
from pyspark.sql.functions import concat, col, lit

hudiTablePartitionKey="route_id"
df1 = df1.withColumn(hudiTablePartitionKey,concat(lit("route_id="),col("route_id")))
df1.select(hudiTablePartitionKey).show(5)

And we can now write out the data to S3. Notice that the Hive Partition Extractor class has changed in the statement below:

```
      .option(HIVE_PARTITION_FIELDS_OPT_KEY, config["partition_keys"])
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,MULTIPART_KEYS_EXTRACTOR_CLASS_OPT_VAL)
      .option(PARTITIONPATH_FIELD_OPT_KEY,"route_id")
```


In [ ]:
(df1.write.format(HUDI_FORMAT)
      .option(PRECOMBINE_FIELD_OPT_KEY, config["sort_key"])
      .option(RECORDKEY_FIELD_OPT_KEY, config["primary_key"])
      .option(TABLE_NAME, config['table_name'])
      .option(OPERATION_OPT_KEY, BULK_INSERT_OPERATION_OPT_VAL)
      .option(BULK_INSERT_PARALLELISM, 6)
      .option(S3_CONSISTENCY_CHECK, "true")
      .option(HIVE_PARTITION_FIELDS_OPT_KEY, config["partition_keys"])
      .option(HIVE_TABLE_OPT_KEY,config['table_name'])
      .option(HIVE_SYNC_ENABLED_OPT_KEY,"true")
      .option(HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY,MULTIPART_KEYS_EXTRACTOR_CLASS_OPT_VAL)
      .option(PARTITIONPATH_FIELD_OPT_KEY,"route_id")
      .mode("Overwrite")
      .save(config['target']))

We can see the partitions fields are present in our Hive table from Glue catalog

```
PARTITIONED BY (`route_id` STRING)
```

Let's now query the data and group by the the partition columns:

In [ ]:
spark.sql("select route_id, count(*) as num_trips from "+config['table_name']+" group by route_id order by route_id").show(20,False)

Let us check the S3 path

REPLACE youraccountID with event engine AWS account ID 

```

$ aws s3 ls s3://mrworkshop-youraccountID-dayone/hudi/hudi_partitioned_trips_table/
                           PRE .hoodie/
                           PRE route_id=A/
                           PRE route_id=B/
                           PRE route_id=C/
                           PRE route_id=D/
                           PRE route_id=E/
                           PRE route_id=F/
                           PRE route_id=G/
                           PRE route_id=H/
                           PRE route_id=I/
                           PRE route_id=J/
2022-03-01 05:50:23          0 .hoodie_$folder$
2022-03-01 05:50:29          0 route_id=A_$folder$
2022-03-01 05:50:29          0 route_id=B_$folder$
2022-03-01 05:50:31          0 route_id=C_$folder$
2022-03-01 05:50:29          0 route_id=D_$folder$
2022-03-01 05:50:29          0 route_id=E_$folder$
2022-03-01 05:50:31          0 route_id=F_$folder$
2022-03-01 05:50:29          0 route_id=G_$folder$
2022-03-01 05:50:29          0 route_id=H_$folder$
2022-03-01 05:50:31          0 route_id=I_$folder$
2022-03-01 05:50:34          0 route_id=J_$folder$
```

Under each partition, there will be partition metadata

REPLACE youraccountID with event engine AWS account ID 

```

$ aws s3 ls s3://mrworkshop-youraccountID-dayone/hudi/hudi_partitioned_trips_table/route_id=A/
2022-03-01 05:50:29         93 .hoodie_partition_metadata
2022-03-01 05:50:32    1730659 b3ea79a8-d0ea-48f2-ae02-3ffea2275744-0_0-245-2900_20220301055022.parquet

```

The other operations Upsert etc. behave the same way on Partitioned tables.